In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D # full connection yapar
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense # full connection yapmaz
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization
import keras

import numpy as np

Using TensorFlow backend.
/Users/emrah.tema/.virtualenvs/jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/emrah.tema/.virtualenvs/jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/emrah.tema/.virtualenvs/jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/e

In [2]:
# 0-255 olan pixel değerlerini 0-1 arasına sığacak şekilde scale eder
train_datagen = image.ImageDataGenerator(
                                        rescale=1./255,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True)

test_datagen = image.ImageDataGenerator(rescale=1./255)

In [7]:
training_set = train_datagen.flow_from_directory('chest_xray/train',
                                                target_size=(64, 64), # resim input shape ile aynı
                                                batch_size=32,
                                                class_mode='binary') # iki sınıfımız var diye binary

Found 5216 images belonging to 2 classes.


In [8]:
test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

Found 624 images belonging to 2 classes.


In [9]:
classifier = Sequential()
classifier.add(Convolution2D(64, (3, 3), input_shape = (64, 64, 3), activation = "relu"))
classifier.add(MaxPooling2D((2,2)))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.15))
classifier.add(Convolution2D(64,(5,5),activation='relu'))
classifier.add(MaxPooling2D((2,2)))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.15))
classifier.add(Convolution2D(128,(3,3),activation='relu'))
classifier.add(MaxPooling2D((2,2)))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.15))
classifier.add(Convolution2D(128,(3,3),activation='relu'))
classifier.add(MaxPooling2D((2,2)))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.15))
classifier.add(GlobalAveragePooling2D())
classifier.add(Dense(units = 1000, activation='relu'))
classifier.add(Dense(units = 1, activation='sigmoid'))



#classifier.add(Convolution2D(64, (3, 3), input_shape = (128, 128, 3), activation = "relu"))
#classifier.add(MaxPooling2D(pool_size = (2, 2)))

#classifier.add(Flatten())
#classifier.add(Dense(units = 32, activation = "relu"))

#classifier.add(Dense(units = 1, activation = "sigmoid"))

classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
classifier.fit_generator(
                        training_set,
                        steps_per_epoch=5216,
                        epochs=3,
                        validation_data=test_set,
                        validation_steps=624)
# val'lı ifadeler test verisinin sonuçları
classifier.save('my_model.h5')

Epoch 1/3
5216/5216 [==============================] - 2713s 520ms/step - loss: 0.1290 - accuracy: 0.9505 - val_loss: 0.3193 - val_accuracy: 0.9006
Epoch 2/3
5216/5216 [==============================] - 2737s 525ms/step - loss: 0.0782 - accuracy: 0.9707 - val_loss: 0.3144 - val_accuracy: 0.8799
Epoch 3/3
2429/5216 [============>.................] - ETA: 22:56 - loss: 0.0639 - accuracy: 0.9759

In [9]:
import numpy as np
from keras.preprocessing import image
import os

In [64]:
classifier.save('my_model.h5')

In [67]:
new_model = keras.models.load_model('my_model.h5')

In [69]:
test_image = image.load_img("chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg", target_size = (128, 128))
test_image = image.img_to_array(test_image)
# resme dördüncü bir boyut eklemek gerekiyor hata almamak için
test_image = np.expand_dims(test_image, axis = 0)
result = new_model.predict(test_image)
print(training_set.class_indices)
print(result[0][0])

{'NORMAL': 0, 'PNEUMONIA': 1}
1.0


In [63]:
counter = 0
c = 0
for file in os.listdir("chest_xray/test/PNEUMONIA"):
    if file.endswith(".jpeg"):
        test_image = image.load_img("chest_xray/test/PNEUMONIA/" + file, target_size = (128, 128))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = classifier.predict(test_image)
        if result[0][0] == 1:
            counter += 1
        c += 1
print(counter, "/", c)
print((1/c)*counter)

308 / 390
0.7897435897435897
